# Bank Customer Churn Prediction Model
### Using Hyperparameter Optimizations in Artificial Neural Network

### Data Preprocessing

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
dataset.shape

(10000, 14)

In [4]:
# As there is no impact of RowNumber, CustomerId and Surname on the model prediction
# So these columns will be removed
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [5]:
# Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [6]:
# Concatenate the Data Frames
X=pd.concat([X,geography,gender],axis=1)

In [7]:
# Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [8]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [9]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [10]:
# In ANN neuron wights get multiplied with input, so it is necessary to scale the inputs to a common scale.
# Also it helps in easier multiplication as I/Ps are scaled down
# It also helps in back propogation as derivatives can be easily found with smaller values.
# As a result, convergence will happen quickly.

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
X_train

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.5698444 ,
        -0.57369368,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ...,  1.75486502,
        -0.57369368, -1.09168714]])

In [12]:
X_test

array([[-0.55204276, -0.36890377,  1.04473698, ...,  1.75486502,
        -0.57369368, -1.09168714],
       [-1.31490297,  0.10961719, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       [ 0.57162971,  0.30102557,  1.04473698, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       ...,
       [-0.74791227, -0.27319958, -1.37744033, ..., -0.5698444 ,
         1.74309049,  0.91601335],
       [-0.00566991, -0.46460796, -0.33936434, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-0.79945688, -0.84742473,  1.04473698, ...,  1.75486502,
        -0.57369368,  0.91601335]])

### Perform Hyperparameter Optimization

In [15]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential 
from keras.layers import Dense, Activation, Embedding, Flatten, BatchNormalization 
from keras.layers import LeakyReLU, PReLU, ELU 
from keras.layers import Dropout 
from keras.activations import sigmoid, relu

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [16]:
def create_model(layers, activation):
    model=Sequential()
    
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes, input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    
    model.add(Dense(units=1, kernel_init='glorot_uniform',activation='sigmoid'))
    
    model.compile(optimizer = 'Adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [17]:
model=KerasClassifier(build_fn=create_model)

In [20]:
layers=[[20],[20,45],[30,60,45],[45,60,50]]
activations=['relu']
param_grid=dict(layers=layers, activation=activations, batch_size=[128,256], epochs=[100])
param_grid

{'layers': [[20], [20, 45], [30, 60, 45], [45, 60, 50]],
 'activation': ['relu'],
 'batch_size': [128, 256],
 'epochs': [100]}

In [21]:
grid=GridSearchCV(estimator=model, param_grid=param_grid)